In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts,GridSearchCV
import statsmodels.api as sm
from sklearn.linear_model import Lasso, LassoCV, ElasticNet, ElasticNetCV,LinearRegression,BayesianRidge,Ridge, LogisticRegression
from sklearn.svm import SVR
import sklearn.metrics as metrics
from sklearn.neighbors import KNeighborsRegressor as KNNR
import scipy
import plotly.plotly as py
import plotly.graph_objs as go
import pygsheets

%matplotlib inline
pd.options.display.max_rows=100
pd.options.display.max_columns=100

- Ideas
    - Bring back data
    - try cleaning it more thoroughly
    - try creating a few more interesting features (look at their distributions!!)
    - Try an MCMC or another approach
    - try PCA then a regression
- Timeline
    - do more extensive data cleaning during the week
    - convert models to DAG

In [6]:
gc = pygsheets.authorize(outh_file='/Users/irarickman/client_secret.json')
file = 'Activity data'
sh = gc.open(file)

wks_1 = sh[0]
strava=wks_1.get_as_df(empty_value=np.nan)
strava.dropna(how='all',inplace=True)

wks_2=sh[1]
whoop=wks_2.get_as_df(empty_value=np.nan)
whoop.dropna(how='all',inplace=True)
whoop.dropna(axis=1,how='all',inplace=True)

wks_3=sh[2]
acts=wks_3.get_as_df(empty_value=np.nan)
acts.dropna(how='all',inplace=True)
acts.dropna(axis=1,how='all',inplace=True)


In [12]:
print(strava.shape)
strava.columns

(267, 44)


Index(['achievement_count', 'athlete_count', 'average_speed', 'average_watts',
       'comment_count', 'commute', 'device_watts', 'distance', 'elapsed_time',
       'elev_high', 'elev_low', 'external_id', 'flagged', 'gear_id',
       'has_kudoed', 'id', 'kilojoules', 'kudos_count', 'manual', 'max_speed',
       'max_watts', 'moving_time', 'name', 'photo_count', 'private',
       'start_date', 'start_date_local', 'timezone', 'total_elevation_gain',
       'total_photo_count', 'trainer', 'type', 'upload_id',
       'weighted_average_watts', 'workout_type', 'miles', 'race',
       'date_string', 'moving_minutes', 'elapsed_minutes', 'rest', 'avg_mph',
       'time_since_last_act', 'order'],
      dtype='object')

In [13]:
strava.type.value_counts()

Run       200
Ride       38
Swim       27
Walk        1
Rowing      1
Name: type, dtype: int64

In [15]:
strava[strava.type=='Run'].describe()

,achievement_count,athlete_count,average_speed,average_watts,comment_count,distance,elapsed_time,elev_high,elev_low,gear_id,id,kilojoules,kudos_count,max_speed,max_watts,moving_time,photo_count,total_elevation_gain,total_photo_count,upload_id,weighted_average_watts,workout_type,miles,race,moving_minutes,elapsed_minutes,rest,avg_mph,time_since_last_act,order
count,200.000000,200.000000,200.000000,0.0,200.000000,200.000000,200.000000,200.000000,200.000000,0.0,2.000000e+02,0.0,200.000000,200.00000,0.0,200.000000,200.0,200.000000,200.00000,2.000000e+02,0.0,13.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,199.000000,200.000000
mean,1.180000,3.705000,3.166715,NaN,0.035000,8910.404500,3225.475000,105.549000,69.149000,NaN,1.977620e+09,NaN,3.520000,6.49000,NaN,2820.780000,0.0,76.505000,0.02500,2.113838e+09,NaN,0.692308,5.536667,0.045000,47.013000,53.757917,6.744917,7.083941,-41.120603,1.115000
std,2.451048,17.703816,0.367820,NaN,0.232476,3985.991326,1391.192388,184.846392,140.123796,NaN,2.563334e+08,NaN,2.648523,2.46042,NaN,1268.368069,0.0,77.834898,0.23377,2.665413e+08,NaN,0.480384,2.476779,0.207824,21.139468,23.186540,8.440639,0.822813,23.853414,0.335167
min,0.000000,1.000000,0.802000,NaN,0.000000,1705.200000,786.000000,-100.200000,-246.800000,NaN,1.483968e+09,NaN,0.000000,3.40000,NaN,439.000000,0.0,8.000000,0.00000,1.598846e+09,NaN,0.000000,1.059562,0.000000,7.316667,13.100000,0.000000,1.794074,-171.000000,1.000000
25%,0.000000,1.000000,3.047000,NaN,0.000000,5890.800000,2191.250000,60.000000,21.000000,NaN,1.768156e+09,NaN,2.000000,4.77500,NaN,1933.250000,0.0,40.000000,0.00000,1.898284e+09,NaN,0.000000,3.660372,0.000000,32.220833,36.520833,2.191667,6.816139,-52.000000,1.000000
50%,0.000000,1.000000,3.183500,NaN,0.000000,8638.200000,3023.000000,68.200000,46.400000,NaN,2.023674e+09,NaN,3.000000,5.75000,NaN,2686.500000,0.0,56.000000,0.00000,2.162670e+09,NaN,1.000000,5.367527,0.000000,44.775000,50.383333,3.525000,7.121490,-37.000000,1.000000
75%,1.000000,1.000000,3.322250,NaN,0.000000,10568.775000,4010.000000,91.600000,55.100000,NaN,2.193670e+09,NaN,5.000000,7.42500,NaN,3394.000000,0.0,81.000000,0.00000,2.337539e+09,NaN,1.000000,6.567130,0.000000,56.566667,66.833333,6.770833,7.431873,-25.000000,1.000000
max,13.000000,181.000000,4.304000,NaN,2.000000,22539.300000,8744.000000,1998.400000,1418.800000,NaN,2.363197e+09,NaN,17.000000,17.40000,NaN,7364.000000,0.0,611.000000,3.00000,2.513811e+09,NaN,1.000000,14.005267,1.000000,122.733333,145.733333,43.100000,9.628048,-1.000000,3.000000
